In [1]:
import torch
import sys
from torchviz import make_dot
import numpy as np

In [2]:
sys.path.append('../')

In [3]:
from ml_variants.conv_lstm.model import Sat2Rad
from ml_variants.conv_lstm.layers.ConvLSTM import ConvLSTM

In [4]:
model = Sat2Rad()

In [12]:
input = torch.ones((2, 16, 16))

In [13]:
input

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1.,

In [14]:
# yhat = model(np.zeros((1, 1)))

In [15]:
myConvLstm = ConvLSTM(
    input_size=(16, 16),
    input_dim=2,
    hidden_dim=[1,1,1],
    kernel_size=(3,3),
    num_layers=3
  )

In [16]:
myConvLstm(input, myConvLstm.get_init_states(1, cuda=False))

RuntimeError: Tensors must have same number of dimensions: got 2 and 4